# pip e imports

In [4]:
import numpy as np
import pandas as pd
import re
from sklearn import preprocessing
from scipy.sparse import coo_matrix, csr_matrix
import requests
from io import BytesIO
from PIL import Image
import matplotlib.pyplot as plt

# Modelo simple 1: en base a lat y long, te recomienda los de mas estrellas y mas review_counts dentro de los que estan cerca de la ubicacion.

In [83]:
pip install folium

   ---------------------------------------- 0.0/108.9 kB ? eta -:--:--
   --- ------------------------------------ 10.2/108.9 kB ? eta -:--:--
   ----------------------------------- -- 102.4/108.9 kB 845.5 kB/s eta 0:00:01
   -------------------------------------- 108.9/108.9 kB 789.2 kB/s eta 0:00:00
   ---------------------------------------- 0.0/85.1 kB ? eta -:--:--
   ---------------------------------------- 85.1/85.1 kB 4.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [84]:
pip install config

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import plotly
import plotly.offline as py
import plotly.graph_objs as go
import plotly.express as px
import hvplot.pandas

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
#from config import MAPBOX_API_KEY

In [7]:
df_business_fil_fil = pd.read_parquet('mi_archivo1.parquet', engine='pyarrow')

In [8]:
df_business_fil_fil.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20997 entries, 3 to 150323
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   business_id   20997 non-null  object 
 1   name          20997 non-null  object 
 2   address       20997 non-null  object 
 3   city          20997 non-null  object 
 4   state         20997 non-null  object 
 5   postal_code   20997 non-null  object 
 6   latitude      20997 non-null  float64
 7   longitude     20997 non-null  float64
 8   stars         20997 non-null  float64
 9   review_count  20997 non-null  int64  
 10  is_open       20997 non-null  int64  
 11  attributes    20787 non-null  object 
 12  categories    20997 non-null  object 
 13  hours         20367 non-null  object 
dtypes: float64(3), int64(2), object(9)
memory usage: 2.4+ MB


In [9]:
top_restaurants_df = df_business_fil_fil.sort_values(by=["stars","review_count"], ascending=False)[:20]

In [10]:
df_business_fil_fil['latitude'].isna().sum()

0

In [11]:
print(f"Número total de filas en el DataFrame: {df_business_fil_fil.shape[0]}")


Número total de filas en el DataFrame: 20997


In [12]:
df_business_location = df_business_fil_fil.dropna(subset=['latitude', 'longitude'])

In [13]:
#Elbow method to determine the number of K in Kmeans Clustering
coords = df_business_location[["latitude", "longitude"]]

inertia = []
K = range(1,25)

# Calculate the inertia for the range of K values
for k in K:
    kmeansModel = KMeans(n_clusters=k)
    kmeansModel = kmeansModel.fit(coords)
    inertia.append(kmeansModel.inertia_)

In [14]:
import matplotlib.pyplot as plt

# Graficar el método del codo
plt.figure(figsize=(10, 6))
plt.plot(K, inertia, 'bo-', color='blue', markersize=8)
plt.xlabel('Número de clústeres (k)')
plt.ylabel('Inercia')
plt.title('Método del Codo para determinar el número óptimo de clústeres')
plt.grid(True)

# Guardar la gráfica en un archivo
plt.savefig('metodo_del_codo.png', dpi=300)

# Opcionalmente, cerrar la figura para liberar memoria
plt.close()


C:\Users\maria\AppData\Local\Temp\ipykernel_20848\2425343386.py:5: UserWarning:

color is redundantly defined by the 'color' keyword argument and the fmt string "bo-" (-> color='b'). The keyword argument will take precedence.



In [16]:
import numpy as np

# Calcular la tasa de cambio relativa de la inercia
rate_of_change = np.diff(inertia) / inertia[:-1]

# Crear un DataFrame para mostrar las tasas de cambio
import pandas as pd
resultados = pd.DataFrame({
    'k': list(K)[:-1],  # Hasta el penúltimo porque np.diff reduce el tamaño
    'inercia': inertia[:-1],
    'tasa_de_cambio': rate_of_change
})

# Imprimir los resultados ordenados
print(resultados)

# Encontrar el índice con la menor reducción significativa
k_optimo = resultados['tasa_de_cambio'].idxmin() + 1  # +1 porque la lista de índices empieza en 0
print(f"El número óptimo de clústeres (k) podría ser: {k_optimo}")

     k       inercia  tasa_de_cambio
0    1  5.295160e+06       -0.771681
1    2  1.208983e+06       -0.539731
2    3  5.564574e+05       -0.457231
3    4  3.020279e+05       -0.251805
4    5  2.259757e+05       -0.469373
5    6  1.199087e+05       -0.106343
6    7  1.071573e+05       -0.529957
7    8  5.036856e+04       -0.531589
8    9  2.359318e+04       -0.466831
9   10  1.257915e+04       -0.953497
10  11  5.849710e+02       -0.224455
11  12  4.536712e+02       -0.076582
12  13  4.189283e+02       -0.042259
13  14  4.012250e+02       -0.282018
14  15  2.880725e+02       -0.027660
15  16  2.801043e+02        0.013335
16  17  2.838394e+02       -0.142305
17  18  2.434475e+02       -0.076441
18  19  2.248381e+02       -0.048247
19  20  2.139903e+02       -0.043375
20  21  2.047085e+02       -0.022094
21  22  2.001857e+02       -0.075587
22  23  1.850542e+02       -0.027731
El número óptimo de clústeres (k) podría ser: 10


In [17]:
kmeans = KMeans(n_clusters= 10, init='k-means++')
kmeans.fit(coords)
y= kmeans.labels_

In [18]:
df_business_location["cluster"]= kmeans.predict(df_business_location[['latitude', 'longitude']])

In [19]:
top_restaurants = df_business_location.sort_values(by=['stars','review_count'],ascending=False)


In [20]:
top_restaurants.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20997 entries, 12307 to 131622
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   business_id   20997 non-null  object 
 1   name          20997 non-null  object 
 2   address       20997 non-null  object 
 3   city          20997 non-null  object 
 4   state         20997 non-null  object 
 5   postal_code   20997 non-null  object 
 6   latitude      20997 non-null  float64
 7   longitude     20997 non-null  float64
 8   stars         20997 non-null  float64
 9   review_count  20997 non-null  int64  
 10  is_open       20997 non-null  int64  
 11  attributes    20787 non-null  object 
 12  categories    20997 non-null  object 
 13  hours         20367 non-null  object 
 14  cluster       20997 non-null  int32  
dtypes: float64(3), int32(1), int64(2), object(9)
memory usage: 2.5+ MB


In [21]:
def recommend_restaurants (df,latitude, longitude):
    # Predict the cluster for longitude and laltiude provided
    cluster = kmeans.predict(np.array([latitude, longitude]).reshape(1, -1))[0]
    print(cluster)
    #Get the best reataurant in this cluster
    return df[df["cluster"]==cluster].iloc[0:5][['name', 'latitude', 'longitude','stars','categories','review_count','business_id']]

In [22]:
recommend_restaurants(top_restaurants,39.97165, -75.12917)

2


c:\Users\maria\Documents\ITBA MARIA\4 1Q\TP2PredictivoAvanzado\.conda\Lib\site-packages\sklearn\base.py:493: UserWarning:

X does not have valid feature names, but KMeans was fitted with feature names



,name,latitude,longitude,stars,categories,review_count,business_id
104287,SUGARED + BRONZED,39.948570,-75.160072,5.0,"Beauty & Spas, Shopping, Waxing, Cosmetics & B...",513,l_7TW_Ix58-QvhQgpJi_Xw
84744,JB Plumbing and Heating,39.955929,-75.157457,5.0,"Plumbing, Water Heater Installation/Repair, Ex...",398,u3zqvp4BYUjzJD7tzx3Jbg
7094,The Sweet Life Bakeshop,39.942203,-75.155223,5.0,"Food, Cupcakes, Desserts, Bakeries",316,qjGS_7iaQDpbVhS6W8qkHQ
102283,Bobby Mack & Co Hair Studio,39.944711,-75.174965,5.0,"Hair Salons, Men's Hair Salons, Hair Stylists,...",291,Og4z8nB4ZMZs3oHkVhB_pA
67317,ICI Macarons & Cafe,39.951910,-75.144381,5.0,"Desserts, Bakeries, Food, Coffee & Tea, Specia...",276,-fs09akgCKv5rTTy7iUHUg


In [23]:
import pickle

# Serializar el modelo de KMeans y los datos
with open('./Data/modelo_recomendacion.pkl', 'wb') as file:
    # Puedes empaquetar el modelo y otros datos necesarios como un diccionario
    pickle.dump({
        'kmeans': kmeans,  # Modelo de clustering
        'df': top_restaurants  # Datos relevantes para las recomendaciones
    }, file)